### 有重跑一次測試（只爬＂兩頁＂）

## 載入用到的套件

In [1]:
import pandas as pd
import requests
from ipywidgets import interact
from bs4 import BeautifulSoup
from collections import defaultdict
from pandas import DataFrame
import re

### Check 需不需要 header

In [2]:
# 聯合網 即時不分類
url_udn = 'https://udn.com/news/breaknews/1/99#breaknews'
requests.get(url_udn).ok    
#my_headers = {'user-agent': 'anything'}    

True

### TEST

In [3]:
"""
article_title, article_link, article_view, article_time = [], [], [], []
for article in article_data:
    try:   # or use `if article.h2 != None`
        article_title.append(article.h2.text.replace('\n',''))
        article_link.append('https://udn.com/' + article.h2.a.get('href'))
        article_view.append(int(article.span.get_text()))
        article_time.append(article.time.get_text())
    except:
        continue
"""

# simplify version??  One line version
#article_title = [article.h2.text.replace('\n', '') for article in article_data if article.h2 != None]
#article_link = ['https://udn.com/' + article.h2.a.get('href').split('?')[0] for article in article_data if article.h2 != None]
#article_view = [int(article.span.get_text()) for article in article_data if article.span != None]
#article_time = [article.time.get_text() for article in article_data if article.time != None] 

"\narticle_title, article_link, article_view, article_time = [], [], [], []\nfor article in article_data:\n    try:   # or use `if article.h2 != None`\n        article_title.append(article.h2.text.replace('\n',''))\n        article_link.append('https://udn.com/' + article.h2.a.get('href'))\n        article_view.append(int(article.span.get_text()))\n        article_time.append(article.time.get_text())\n    except:\n        continue\n"

---
## 爬取資料
### 基於該網頁為滾軸式（infinity scroll），因此要用動態爬蟲

### Selenium scroll down and find target
ChromeDriver會有版本與Chrome不同的問題，會影響程式的執行，目前採取手動更新版本的方式解決，以下也有附上一些網路上找到的解決方案  
- ChromeDriver 版本自動更新之解決方案
  * https://medium.com/drunk-wis/python-selenium-chrome-browser-%E8%88%87-driver-%E6%83%B1%E4%BA%BA%E7%9A%84%E7%89%88%E6%9C%AC%E7%AE%A1%E7%90%86-cbaf1d1861ce
  * https://sqa.stackexchange.com/questions/41928/how-to-autoupdate-chrome-driver-in-selenium  
    
    
- ChromeDriver 手動更新版本（下載位置）  
  https://chromedriver.storage.googleapis.com/index.html  
若ChromeDriver和Chrome版本不同，需下載與Chrome相同版本之ChromeDriver執行檔，並將  
`driver = webdriver.Chrome(executable_path = "./chromedriver.exe", chrome_options = chrome_options)`  
中執行路徑（`executable_path`）更改為，與目前電腦中Chrome的版本相同之ChromeDriver執行檔，其所在路徑  
最後一次執行程式碼時，Chrome版本為98.0.4758.102，因此在此附上的ChromeDriver的版本也是如此

In [4]:
# https://ithelp.ithome.com.tw/articles/10204231
# https://www.youtube.com/watch?v=aqXkKgkyk2I
# https://stackoverflow.com/questions/22130109/cant-use-chrome-driver-for-selenium
# ----------------------------------------
# https://www.learncodewithmike.com/2020/05/python-selenium-scraper.html
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time


while True:
    page = input('要抓到第幾"頁"？，請輸入一整數')
    try:
        page = int(page)
        break
    except:
        print("輸入錯誤，請重新輸入")
    
    
# https://blog.csdn.net/cool_soup29/article/details/90412610?utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-1.control&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-1.control
# 禁止 chrome 彈出顯示通知的視窗
chrome_options = Options()
#chrome_options.add_argument('--headless')
prefs = {'profile.default_content_setting_values':{'notifications': 2}}
chrome_options.add_experimental_option('prefs', prefs)
    
driver = webdriver.Chrome(executable_path = "./chromedriver.exe", options = chrome_options)
driver.get("https://udn.com/news/breaknews/1/99#breaknews")
driver.maximize_window()


while True:
    try:
        if page == 1:
            xpath = "//*[@id='breaknews']/div[1]/div[1]/div[2]/h2/a"
        else:
            number = 1 + 20*(page-2)
            xpath = "//*[@id='breaknews']/div[1]/section/div[{}]/div[2]/h2/a".format(number)
        target = driver.find_element_by_xpath(xpath)
        driver.execute_script("arguments[0].scrollIntoView();", target)
        break
    except:
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight-100)")
        time.sleep(5)
        
        


要抓到第幾"頁"？，請輸入一整數2


### BeautifulSoup scraping data + data processing

In [5]:
# 原本爬蟲只差在這兩行 (只能跑第一頁共20個文章)
# source = requests.get(url_udn).text #, headers = my_headers).text
# soup = BeautifulSoup(source, 'lxml')   # 像是把 source 分段的感覺?
# 配合 selenium scroll down 爬蟲
# https://stackoverflow.com/questions/50595558/selenium-scrolling-and-scraping-with-beautifulsoup-produces-duplicate-results

soup = BeautifulSoup(driver.page_source, 'lxml')  
driver.close()
article_data = soup.find_all('div', class_ = 'story-list__text')
article_title = [article.h2.text.replace('\n', '') for article in article_data if article.h2 != None]
article_link = ['https://udn.com/' + article.h2.a.get('href').split('?')[0] for article in article_data if article.h2 != None]
article_view = [int(article.span.get_text()) for article in article_data if article.span != None]
article_time = [article.time.get_text() for article in article_data if article.time != None] 

false_link = []
article_content, article_tag, article_subtag = [], [], []
for link in article_link:
    
    # 聯合報有夠誇張文章連結給你直接跳轉到聯合報 vip 的網站(還要修正一次連結), 文章需要訂閱($)才能爬完整, tag方式也不一樣 030
    subsoup = BeautifulSoup(requests.get(link).text, 'lxml')
    
    # 連結會跳轉的情況
    try:
        link = subsoup.find('section', class_ = 'article-content__wrapper').script.string.split("\"")[1].split('?')[0]
        # https://stackoverflow.com/questions/26192727/extract-content-of-script-with-beautifulsoup
        subsoup = BeautifulSoup(requests.get(link).text, 'lxml')
        article_tag.append([subsoup.find('a', class_ = 'article-content__cate').text])
        article_content.append([subsoup.find('section', class_ = 'article-content__editor').text])
        
        # 有可能沒有 subtag (keyword)
        try:
            article_subtag.append([subsoup.find('section', class_ = 'keywords').text.replace('\n', '/')])
        except:
            article_subtag.append([])
    
    # 和原連結相同的情況
    # 這裡內部寫很多 try, except是因為，文章的格式都不太一樣（這也是超級麻煩的），因此文章所在的html位置也不一樣
    # 因此要考慮很多狀況（當初忘了有甚麼狀況了XD），但盡管考慮那麼多，還是有部分文章沒辦法用，也懶得一個一個找了，所以找不到的都設沒內容
    except:
        # subsoup.select('nav[class="article-content__breadcrumb"]') == subsoup.find_all('nav', class_ = 'article-content__breadcrumb')
        try:
            article_tag.append([tag.get_text().replace('\n', '/') for tag in subsoup.find_all('nav', class_ = 'article-content__breadcrumb')])
        except:
            articl_tag.append([])
        try:
            article_content.append([content.get_text() for content in subsoup.find('section', class_ = 'article-content__editor').find_all('p') if content != None])
        except:
            try:
                article_content.append(subsoup.find('section', class_ = 'article-content__wrapper').find_all('p'))
            except:
                driver = webdriver.Chrome(executable_path = "./chromedriver.exe", options = chrome_options)
                driver.get(link)
                driver.execute_script("window.scrollBy(0, document.body.scrollHeight)")
                subsoup = BeautifulSoup(driver.page_source, 'lxml')
                driver.close()
                try:
                    article_content.append([[content.get_text() for content in subsoup.find('section', class_ = 'article-holder').find_all('p')]])
                except:
                    try:
                        article_content.append([content.get_text() for content in subsoup.find_all('p')])
                    except:
                        article_content.append([])
                        false_link.append(link)
                        
        # 因為抓content會把相關新聞和快報什麼的抓進來, 因此要做進一步處理
        for i in range(len(article_content)):
            if len(article_content[i]) > 7 and '專題' in article_content[i][-7]:
                article_content[i] = article_content[i][:-7]
        
        # 有可能沒有 subtag (keyword)
        try:
            article_subtag.append([subsoup.find('section', class_ = 'keywords').text.replace('\n', '/')])
        except:
            article_subtag.append([])


---
## CHECK 用（當初寫的，沒再做更動，所以很亂）（請忽略）

### 確認文章標題編號

In [6]:
"""
for i,j in enumerate(article_title):
    if "救命技術" in j:/
        print(i,j)
print(article_link[333])
"""
print('')

### 確認該文章內文所在位置

In [7]:
"""
article_content, article_tag, article_subtag = [], [], []
link = article_link[378]
driver2 = webdriver.Chrome(executable_path = "D:\Program Files\chromedriver\chromedriver.exe", chrome_options = chrome_options)
driver2.get(link)
driver2.execute_script("window.scrollBy(0, document.body.scrollHeight)")
subsoup28 = BeautifulSoup(driver2.page_source, 'lxml')
subsoup28.find_all('section', class_ = 'story_body_content)
"""
print('')

### 當初用錯所以重用: tag subtag content歸零重新開始再算一次

In [8]:
"""
article_content, article_tag, article_subtag = [], [], []
for link in article_link:
    
    # 聯合報有夠誇張文章連結給你直接跳轉到聯合報 vip 的網站(還要修正一次連結), 文章需要訂閱($)才能爬完整, tag方式也不一樣 030
    subsoup = BeautifulSoup(requests.get(link).text, 'lxml')
    try:
        link = subsoup.find('section', class_ = 'article-content__wrapper').script.string.split("\"")[1].split('?')[0]
        # https://stackoverflow.com/questions/26192727/extract-content-of-script-with-beautifulsoup
        subsoup = BeautifulSoup(requests.get(link).text, 'lxml')
        article_tag.append([subsoup.find('a', class_ = 'article-content__cate').text])
        article_content.append([subsoup.find('section', class_ = 'article-content__editor').text])
        
        # 有可能沒有 subtag (keyword)
        try:
            article_subtag.append([subsoup.find('section', class_ = 'keywords').text.replace('\n', '/')])
        except:
            article_subtag.append([])
            
    except:
        # subsoup.select('nav[class="article-content__breadcrumb"]') == subsoup.find_all('nav', class_ = 'article-content__breadcrumb')
        try:
            article_tag.append([tag.get_text().replace('\n', '/') for tag in subsoup.find_all('nav', class_ = 'article-content__breadcrumb')])
        except:
            articl_tag.append([])
        try:
            article_content.append([content.get_text() for content in subsoup.find('section', class_ = 'article-content__editor').find_all('p') if content != None])
        except:
            try:
                article_content.append(subsoup.find('section', class_ = 'article-content__wrapper').find_all('p'))
            except:
                driver2 = webdriver.Chrome(executable_path = "D:\Program Files\chromedriver\chromedriver.exe", chrome_options = chrome_options)
                driver2.get(link)
                driver2.execute_script("window.scrollBy(0, document.body.scrollHeight)")
                subsoup = BeautifulSoup(driver2.page_source, 'lxml')
                driver2.close()
                try:
                    article_content.append([content.get_text() for content in subsoup.find('section', class_ = 'article-holder').find_all('p')])   # 原本是[[]]
                except:
                    try:
                        article_content.append([content.get_text() for content in subsoup.find_all('p')])
                    except:
                        article_content.append([])
                        false_link.append(link)
        # 因為抓content會把相關新聞和快報什麼的抓進來, 因此要做進一步處理
        for i in range(len(article_content)):
            if len(article_content[i]) > 7 and '專題' in article_content[i][-7]:
                article_content[i] = article_content[i][:-7]
        
        # 有可能沒有 subtag (keyword)
        try:
            article_subtag.append([subsoup.find('section', class_ = 'keywords').text.replace('\n', '/')])
        except:
            article_subtag.append([])
"""
print('')

---
## Create DataFrame

### Check if length of columns are the same

In [9]:
print(len(article_link), len(article_tag), len(article_subtag), len(article_time), len(article_view), len(article_title), len(article_content))

40 40 40 40 40 40 40


### 如果長度不一，確認缺失值並加入對應欄位裡

In [10]:
#article_content.insert(j,'')
#subsoup = BeautifulSoup(requests.get(article_link[2665]).text, 'lxml')
#try:
#    link = subsoup.find('section', class_ = 'article-content__wrapper').script.string.split("\"")[1].split('?')[0]
#    subsoup = BeautifulSoup(requests.get(link).text, 'lxml')
#    print('article_tag:', [subsoup.find('a', class_ = 'article-content__cate').text])
#    print('article_content', [subsoup.find('section', class_ = 'article-content__editor').text])
#    try:
#        print('article_subtag', [subsoup.find('section', class_ = 'keywords').text.replace('\n', '/')])
#    except:
#        print('article_subtag', [])
#            
#except:
#    try:
#        print('article_tag', [tag.get_text().replace('\n', '/') for tag in subsoup.find_all('nav', class_ = 'article-content__breadcrumb')])
#    except:
#        print('article_tag', [])
#    try:
#        print('article_content', [content.get_text() for content in subsoup.find('section', class_ = 'article-content__editor').find_all('p') if content != None])
#    except:
#        try:
#            print('article_content', subsoup.find('section', class_ = 'article-content__wrapper').find_all('p'))
#        except:
#            driver2 = webdriver.Chrome(executable_path = "D:\Program Files\chromedriver\chromedriver.exe", chrome_options = chrome_options)
#            driver2.get(article_link[2665])
#            driver2.execute_script("window.scrollBy(0, document.body.scrollHeight)")
#            subsoup = BeautifulSoup(driver2.page_source, 'lxml')
#            driver2.close()
#            try:
#                 print('article_content', [content.get_text() for content in subsoup.find('section', class_ = 'article-holder').find_all('p')])  # 原本是 [[]]
#            except:
#                try:
#                    print('article_content', [content.get_text() for content in subsoup.find_all('p')])
#                except:
#                    print('article_content', [])
#        # 因為抓content會把相關新聞和快報什麼的抓進來, 因此要做進一步處理
#        
#        # 有可能沒有 subtag (keyword)
#    try:
#        print('article_subtag', [subsoup.find('section', class_ = 'keywords').text.replace('\n', '/')])
#    except:
#        print('article_subtag', [])

### 建立 DataFrame

In [11]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html
pd.set_option("max_colwidth", 40)
# https://stackoverflow.com/questions/25351968/how-to-display-full-non-truncated-dataframe-information-in-html-when-convertin
df = pd.DataFrame({'標題': article_title, 'tag': article_tag , 'subtag': article_subtag, 
                   '時間': article_time, '觀看人數': article_view, '內文': article_content, '連結': article_link})
df

,標題,tag,subtag,時間,觀看人數,內文,連結
0,婦人於社區SPA池溺斃 管理公司判賠121萬餘元,[/udn/社會/法律前線/],[/SPA/溺水/],2022-03-12 16:05,0,[新北市劉姓婦人於自家社區SPA池溺斃，案經家屬求償，士院認為，受委任管...,https://udn.com//news/story/7321/615...
1,李克強10年總理路 成維繫改革開放路線樞紐角色,[/udn/兩岸/兩岸要聞/],[/李克強/中共/],2022-03-12 16:05,6,[中國國務院總理李克強的10年任期將到尾聲。學者張弘遠說，李克強任內持續...,https://udn.com//news/story/7331/615...
2,談觀光振興 日相岸田文雄：為重啟振興方案做好準備,[/udn/全球/國際焦點/],[/日本/岸田文雄/觀光/新冠肺炎/],2022-03-12 16:03,20,"[, \n, \n日本首相岸田文雄今天前往宮城縣石卷市訪視，在答覆記者採...",https://udn.com//news/story/6809/615...
3,愛心特展「 施情畫義」閉幕 319件作品抽籤決定有緣人,[/udn/文教/中小學/],[/愛心/台電/苗栗縣/],2022-03-12 16:03,6,"[, \n苗栗縣第7屆「施情畫義、千動愛心」美術特展今天閉幕，展出的31...",https://udn.com//news/story/6898/615...
4,俄侵烏克蘭效應 立陶宛總理加入步槍兵聯盟,[/udn/全球/俄烏各界因應/],[/烏克蘭/俄羅斯/俄烏戰爭/立陶宛/],2022-03-12 16:02,85,"[, \n俄羅斯入侵烏克蘭後，立陶宛出現一波申請加入步槍兵聯盟（Rifl...",https://udn.com//news/story/122699/6...
5,台南購物節2棟豪宅抽出 爽！家樂福花300元中830萬公寓,[/udn/生活/生活新聞/],[/台南/],2022-03-12 16:02,18,[台南市政府下午在高鐵特定區三井OUTLET PARK 舉辦台南購物節最...,https://udn.com//news/story/7266/615...
6,朱立倫受訪談藍白合 柯文哲：這樣約見面太做作,[/udn/要聞/政治/],[/柯文哲/高雄市/朱立倫/],2022-03-12 16:02,23,[民眾黨在北中南舉辦三場華山論壇，第一場選在高雄登場。台北市長柯文哲說，...,https://udn.com//news/story/6656/615...
7,突遭打腳不忍了？大象反擊踢飛馴獸師「往死裡踩」影片曝光,[/udn/生活/動物星球/],[/大象/印度/],2022-03-12 16:00,40,[在印度不少傳統節慶，會看到大象表演的橋段，然而台下的情況可能不如大家看...,https://udn.com//news/story/7470/615...
8,Live／HBL四強激戰 淡商挑戰三連霸要過陽明張聿嵐這關,[],[],2022-03-12 15:58,1,[],https://udn.com//news/story/8688/615...
9,德國之聲專訪／「817萬人不用腦筋」 洪秀柱：民進黨害台灣走入絕境,[/udn/要聞/政治/],[/國民黨/洪秀柱/],2022-03-12 15:57,90,"[, \n, \n國民黨前主席洪秀柱今年2月與中國對台第二把手、中國全國...",https://udn.com//news/story/6656/615...


---
## 存檔

In [12]:
# df.to_csv("dfsave.csv")